In [1]:
# %matplotlib notebook
%matplotlib inline

# %connect_info
import flow.misc
text = """
Jupyter connect hash:
{word}

To connect to current notebook:
    $> jupyter console --existing {filename}
    
"""
print(text.format(word=flow.misc.notebook_word(), filename=flow.misc.notebook_file(flow.misc.notebook_word())))
print("Last run: {}".format(flow.misc.timestamp()))


Jupyter connect hash:
electrical

To connect to current notebook:
    $> jupyter console --existing kernel-eca70eb9-c996-42fd-87bd-fa225239d043.json
    

Last run: 2018-10-29-15h56m51s


/usr/local/lib/python2.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.23) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(context='notebook', style='darkgrid')
# sns.set(context='talk', style='darkgrid')

import flow
import jzap

## Testing GLM results

In [9]:
date = flow.metadata.Date('OA205', 180926)
glm = date.glm()
date

Date(mouse=OA205, date=180926, tags=(u'jeff',), photometry=())

In [10]:
glm.groups()

['plus_correct',
 'plus_correct',
 'plus_correct',
 'plus_correct',
 'plus_correct',
 'plus_correct',
 'plus_correct',
 'plus_correct',
 'plus_miss',
 'plus_miss',
 'plus_miss',
 'plus_miss',
 'plus_miss',
 'plus_miss',
 'plus_miss',
 'plus_miss',
 'minus_correct',
 'minus_correct',
 'minus_correct',
 'minus_correct',
 'minus_correct',
 'minus_correct',
 'minus_correct',
 'minus_correct',
 'minus_miss',
 'minus_miss',
 'minus_miss',
 'minus_miss',
 'minus_miss',
 'minus_miss',
 'minus_miss',
 'minus_miss',
 'neutral_correct',
 'neutral_correct',
 'neutral_correct',
 'neutral_correct',
 'neutral_correct',
 'neutral_correct',
 'neutral_correct',
 'neutral_correct',
 'neutral_miss',
 'neutral_miss',
 'neutral_miss',
 'neutral_miss',
 'neutral_miss',
 'neutral_miss',
 'neutral_miss',
 'neutral_miss',
 'offset',
 'offset',
 'offset',
 'offset',
 'offset',
 'offset',
 'offset',
 'offset',
 'lick_onsets',
 'lick_onsets',
 'lick_onsets',
 'lick_onsets',
 'lick_onsets',
 'lick_onsets',
 'lick_o

In [5]:
glm.labels()

IndexError: too many indices for array

In [ ]:
date.runs()[1].trace2p().cstraces('reward').

In [11]:
(1500-873)*2

1254

In [12]:
(1500-571)*2

1858